In [1]:
import numpy as np
import pandas as pd
import os
import glob
from helper import get_data_splits, id_key, get_model

In [2]:
# Below identifiers correspond to evaluation protocol 1, 2 and 3 respectively
# identifiers - "TouchPose_crossSession", "TouchPose_crossSubjects", "TouchPose_crossGestures"
identifier = "TouchPose_crossSession"
data_path = "../data/"
model_path = "../models/"

In [3]:
files = glob.glob(data_path+'*.pkl')
df = pd.concat([pd.read_pickle(fp) for fp in files], ignore_index=True)

In [4]:
control = id_key[identifier]
df_group = df.groupby([control])["cap_img","depth_img"]
g_id = list(df_group.groups.keys())

/usr/local/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [5]:
depth_err = []
for i in range(len(g_id)):
    test_set = [g_id[i]]
    X_test_cap, Y_test_depth= get_data_splits(df_group,test_set)
    model = get_model(identifier, model_path, test_set)
    Y_out_depth = model.predict(X_test_cap, batch_size=128)
    depth_error = np.mean(abs(Y_test_depth - Y_out_depth[:,:,:,0])) * 255
    depth_err.append(depth_error)
    print("Accuracy", g_id[i],depth_error)

Accuracy right_hand_1 20.406749270604536
Accuracy right_hand_2 18.964988451302325
Accuracy right_hand_3 22.77970285068013


In [6]:
print(identifier)
print("\t Depth Error",np.mean(depth_err),"SD",np.std(depth_err))

TouchPose_crossSession
	 Depth Error 20.717146857529 SD 1.5727410220549112
